In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools
from scipy.special import kn
from scipy import optimize
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data

In [24]:
fname = '/Users/theoares/Dropbox (MIT)/research/0nubb/analysis_output/24I/ml0p01/Z_gamma_MOM4.h5'
f = h5py.File(fname, 'r')
Zq = np.real(f['Zq'][()][0])
ZV = np.real(f['ZV'][()][0])
ZA = np.real(f['ZA'][()][0])
Z = np.zeros((len(Zq), 5, 5), dtype = np.float64)
for i, j in itertools.product(range(5), repeat = 2):
    key = 'Z' + str(i + 1) + str(j + 1)
    Z[:, i, j] = np.real(f[key][()])

In [26]:
Z.shape

(50, 5, 5)

In [27]:
print('Zq = ' + str(np.mean(Zq)) + ' pm ' + str(np.std(Zq, ddof = 1)))
print('ZV = ' + str(np.mean(ZV)) + ' pm ' + str(np.std(ZV, ddof = 1)))
print('ZA = ' + str(np.mean(ZA)) + ' pm ' + str(np.std(ZA, ddof = 1)))
print('Z_ij = ' + str(np.mean(Z, axis = 0)) + ' pm ' + str(np.std(Z, ddof = 1, axis = 0)))

Zq = 1.0844177 pm 0.00045575234
ZV = 0.95903784 pm 0.00014591271
ZA = 0.9593879 pm 0.00014191719
Z_ij = [[ 9.65003254e-01 -5.03979229e-04 -1.84725544e-04  2.52861608e-04
   1.44526220e-05]
 [-3.95131282e-04  1.10892017e+00  3.17617817e-01 -9.96618513e-04
  -8.00521210e-05]
 [ 3.08825359e-05  3.56414329e-02  8.65313221e-01  2.50090904e-03
  -4.02011111e-05]
 [ 1.71937206e-05 -1.69354991e-05  2.25564730e-03  9.16734830e-01
  -3.38001417e-02]
 [ 5.32962438e-05 -9.78537067e-05  7.04702530e-04 -2.94647839e-01
   1.12477435e+00]] pm [[3.96383387e-04 2.02842174e-05 1.07432671e-05 9.96086431e-06
  4.93360474e-06]
 [1.35984019e-05 6.81576090e-04 5.35053940e-04 4.91712174e-05
  1.91320243e-05]
 [4.74941543e-06 8.43872566e-05 3.75755641e-04 1.03928824e-04
  1.09235417e-05]
 [1.78150691e-06 8.43142068e-06 1.00776576e-04 4.31226172e-04
  8.25649890e-05]
 [3.04594146e-06 1.37881319e-05 2.78091514e-05 5.56543228e-04
  6.70118269e-04]]


In [28]:
# Extract components of Z_{ij} with operator mixing
# Running effects seem to be much larger for the last 3 components than for the first 2, which could explain why the 
# first few are much closer to 1. I'm interested to see what happens when I run these to the matching point and 
# convert to MSbar, I expect they'll be closer to 1. (i.e. see Figures 7 - 10 in the Kaon mixing paper.)
print('(27, 1)')
print('Z_11 = ' + str(np.mean(Z[:, 0, 0])) + ' pm ' + str(np.std(Z[:, 0, 0], ddof = 1)))
print('\n(8, 8)')
print('Z_22 = ' + str(np.mean(Z[:, 1, 1])) + ' pm ' + str(np.std(Z[:, 1, 1], ddof = 1)))
print('Z_23 = ' + str(np.mean(Z[:, 1, 2])) + ' pm ' + str(np.std(Z[:, 1, 2], ddof = 1)))
print('Z_32 = ' + str(np.mean(Z[:, 2, 1])) + ' pm ' + str(np.std(Z[:, 2, 1], ddof = 1)))
print('Z_33 = ' + str(np.mean(Z[:, 2, 2])) + ' pm ' + str(np.std(Z[:, 2, 2], ddof = 1)))
print('\n(6, bar{6})')
print('Z_44 = ' + str(np.mean(Z[:, 3, 3])) + ' pm ' + str(np.std(Z[:, 3, 3], ddof = 1)))
print('Z_45 = ' + str(np.mean(Z[:, 3, 4])) + ' pm ' + str(np.std(Z[:, 3, 4], ddof = 1)))
print('Z_54 = ' + str(np.mean(Z[:, 4, 3])) + ' pm ' + str(np.std(Z[:, 4, 3], ddof = 1)))
print('Z_55 = ' + str(np.mean(Z[:, 4, 4])) + ' pm ' + str(np.std(Z[:, 4, 4], ddof = 1)))

(27, 1)
Z_11 = 0.9650032544136047 pm 0.000396383386509703

(8, 8)
Z_22 = 1.1089201736450196 pm 0.0006815760904604431
Z_23 = 0.3176178169250488 pm 0.0005350539401416135
Z_32 = 0.03564143292605877 pm 8.43872566110794e-05
Z_33 = 0.8653132212162018 pm 0.00037575564148033226

(6, bar{6})
Z_44 = 0.9167348301410675 pm 0.00043122617203798254
Z_45 = -0.033800141662359236 pm 8.256498903665052e-05
Z_54 = -0.2946478390693665 pm 0.0005565432278159799
Z_55 = 1.124774351119995 pm 0.0006701182693824168


In [12]:
def get_std_range(data):
    n_samples = len(data)
    sample_range = range(2, len(data))
    errors = np.zeros(len(sample_range), dtype = np.float64)
    for idx, n in enumerate(sample_range):
        subdata = data[:n]
        subdata_boot = bootstrap(subdata, Nb = 10)
        errors[idx] = np.std(subdata_boot, ddof = 1)
    return errors

In [22]:
Zq_std = get_std_range(Zq)
print('Error variation')
print(Zq_std)
print('\nRaw data for Zq')
print(Zq)

Error variation
[0.00426734 0.00655951 0.00477323 0.0103822  0.00732325 0.0074208
 0.01337164 0.01143453 0.00654077 0.00762282 0.00431678]

Raw data for Zq
[1.1463767 1.1571964 1.1320938 1.1418558 1.0829463 1.1042217 1.16459
 1.0936686 1.1187632 1.0975311 1.143033  1.1054981 1.1704788]


In [20]:
Z11_std = get_std_range(Z[:, 0, 0])
print('Error variation')
print(Z11_std)
print('\nRaw data for Z11')
print(Z[:, 0, 0])

Error variation
[0.00987879 0.0193413  0.01554036 0.01714636 0.01229979 0.0144502
 0.02198467 0.02020836 0.01150502 0.01210305 0.00703886]

Raw data for Z11
[1.00449467 1.02954197 0.95184058 0.97307116 0.89899749 0.91992623
 1.03037477 0.91762978 0.94866258 0.92881095 1.01584899 0.93361229
 0.98836869]


In [23]:
Z22_std = get_std_range(Z[:, 1, 1])
print('Error variation')
print(Z22_std)
print('\nRaw data for Z22')
print(Z[:, 1, 1])

Error variation
[0.02191894 0.032085   0.02281209 0.01976816 0.01490004 0.01942518
 0.02438198 0.0253465  0.01624331 0.01129509 0.00902434]

Raw data for Z22
[1.04440701 1.09998167 0.97874707 1.02788234 0.94621605 0.95653564
 1.10393548 0.97646785 1.00751805 0.96715873 1.04393137 0.99386007
 1.06451702]
